In [1]:
import random
import time

import numpy as np
import pandas as pd
import torch
from torch import nn

import evaluate
from datasets import load_dataset
from peft import (get_peft_model,
                  LoraConfig,
                  PrefixTuningConfig,
                  TaskType,)
from transformers import (AutoModelForSequenceClassification,
                          AutoTokenizer,
                          DataCollatorWithPadding,
                          Trainer, 
                          TrainingArguments,)

# Фиксация сидов для воспроизводимости
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# Устройство
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## 0. Загрузка/подготовка

In [2]:
dataset = load_dataset("dair-ai/emotion")

In [3]:
model_checkpoint = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [4]:
def tokenize(example):
    return tokenizer(example["text"], truncation=True)

In [5]:
dataset = dataset.map(tokenize, batched=True)

In [6]:
label_names = dataset['train'].features['label'].names
num_labels = len(label_names)
label_names

['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']

## 1. Метрики до дообучения

In [7]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
accuracy_metric  = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    return {
        "accuracy": accuracy_metric.compute(predictions=preds, references=labels)["accuracy"],
        "precision": precision_metric.compute(predictions=preds, references=labels, average="macro", zero_division=0)["precision"],
        "recall": recall_metric.compute(predictions=preds, references=labels, average="macro", zero_division=0)["recall"],
        "f1_macro": f1_metric.compute(predictions=preds, references=labels, average="macro")["f1"],
    }

In [9]:
training_args = TrainingArguments(
    output_dir="./pretrain",
    eval_strategy="epoch",
    per_device_eval_batch_size=16,
)

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=dataset['validation'],
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [11]:
trainer.evaluate(dataset['test'])

{'eval_loss': 1.6839704513549805,
 'eval_model_preparation_time': 0.0,
 'eval_accuracy': 0.151,
 'eval_precision': 0.14487953510576054,
 'eval_recall': 0.1618697266408435,
 'eval_f1_macro': 0.07286239293441214,
 'eval_runtime': 2.107,
 'eval_samples_per_second': 949.235,
 'eval_steps_per_second': 59.327}

In [12]:
torch.cuda.memory_allocated() / (1024 ** 2)

426.87255859375

## 2. Full finetuning

In [13]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
def train_with_memory_and_metrics(trainer):
    num_params = sum(p.numel() for p in trainer.model.parameters() if p.requires_grad)

    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    start = time.time()
    trainer.train()
    elapsed = time.time() - start

    results = trainer.evaluate(dataset["test"])

    mem_mb = torch.cuda.max_memory_allocated() / (1024 ** 2)

    print(f"Количество обучаемых параметров: {num_params}")
    print(f"Время: {elapsed:.2f} s")
    print(f"Метрики: {results}")
    print(f"Использование памяти gpu: {mem_mb:.2f} MB")

    return {
        "metrics": results,
        "time_s": elapsed,
        "mem_mb": mem_mb,
        "num_params": num_params
    }

Аргументы трейнера выбраны одинаковыми для всех видов дообучения для сравнения:
- оптимизатор AdamW (по умолчанию в HF Trainer)
- lr=2e-5 — стандарт для дообучения BERT-подобных
- batch_size=16 — для укладывания в память
- epochs=15 — даёт стабильную сходимость в данном случае

In [ ]:
training_args = TrainingArguments(
    output_dir="./full_finetining",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    weight_decay=0.01,
    seed=SEED,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset['validation'],
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [16]:
full_finetining_metrics = train_with_memory_and_metrics(trainer)

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 Macro
1,0.228400,0.199299,0.929000,0.896605,0.917308,0.903498
2,0.147000,0.158420,0.934500,0.927437,0.891392,0.906706
3,0.110800,0.151150,0.942500,0.913778,0.926093,0.919284
4,0.105100,0.195490,0.939500,0.921116,0.918304,0.918506
5,0.067800,0.269534,0.934500,0.916425,0.907580,0.911728
6,0.048900,0.312168,0.932500,0.911735,0.905060,0.908158
7,0.036300,0.298440,0.940500,0.923355,0.914100,0.918547
8,0.028900,0.396841,0.938500,0.922220,0.912749,0.917302
9,0.012600,0.367114,0.939000,0.927469,0.910057,0.917900
10,0.022400,0.381348,0.938500,0.924143,0.909212,0.915893


Количество обучаемых параметров: 109486854
Время: 1453.84 s
Метрики: {'eval_loss': 0.4773561954498291, 'eval_accuracy': 0.9295, 'eval_precision': 0.8865127398653881, 'eval_recall': 0.885334840654208, 'eval_f1_macro': 0.8857581442879335, 'eval_runtime': 2.0, 'eval_samples_per_second': 999.985, 'eval_steps_per_second': 62.499, 'epoch': 15.0}
Использование памяти gpu: 2153.35 MB


## 3. Linear probing

LayerNorm → Linear → ReLU → Dropout → LayerNorm → Linear
- Простая feed-forward сеть с двумя линейными проекциями, нормализацией и дропаутом, Dropout и LayerNorm помогают стабилизировать обучение и предотвратить переобучение в условиях малого числа обучаемых параметров.

In [17]:
class LinearProbeHead(torch.nn.Module):
    def __init__(self, hidden_size, num_labels):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.LayerNorm(hidden_size),
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.LayerNorm(hidden_size // 2),
            nn.Linear(hidden_size // 2, num_labels)
        )

    def forward(self, x):
        return self.classifier(x)

In [18]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
model.classifier = LinearProbeHead(hidden_size=model.config.hidden_size, num_labels=num_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# freeze BERT: обучаем только голову (~300k параметров) для быстрой отладки
for name, param in model.named_parameters():
    if not name.startswith("classifier"):
        param.requires_grad = False

In [20]:
training_args = TrainingArguments(
    output_dir="./linear_probing",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    weight_decay=0.01,
    seed=SEED,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset['validation'],
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [21]:
linear_probing_metrics = train_with_memory_and_metrics(trainer)

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 Macro
1,1.572400,1.535964,0.427000,0.143386,0.227367,0.174668
2,1.534000,1.481792,0.450000,0.147962,0.235142,0.181445
3,1.503500,1.454042,0.475500,0.156435,0.248939,0.192047
4,1.478700,1.431136,0.480500,0.260956,0.256108,0.200059
5,1.462000,1.403598,0.471000,0.287460,0.257093,0.207948
6,1.450500,1.410270,0.486500,0.275933,0.260057,0.201708
7,1.447000,1.384009,0.501500,0.315395,0.267122,0.213717
8,1.429300,1.360353,0.499000,0.295384,0.267785,0.215956
9,1.431800,1.360855,0.504000,0.507574,0.279196,0.237196
10,1.431500,1.367918,0.498000,0.482270,0.273465,0.225655


Количество обучаемых параметров: 299910
Время: 359.18 s
Метрики: {'eval_loss': 1.310072660446167, 'eval_accuracy': 0.503, 'eval_precision': 0.405377564086365, 'eval_recall': 0.2717470693308338, 'eval_f1_macro': 0.23209638260975293, 'eval_runtime': 2.0317, 'eval_samples_per_second': 984.413, 'eval_steps_per_second': 61.526, 'epoch': 15.0}
Использование памяти gpu: 1274.20 MB


## 4. Prefix tuning

Выбрала Prefix tuning:
- Prefix Tuning создаёт виртуальные ключи (K) и значения (V) для механизма self-attention
  на каждом слое. Это даёт более «глубокий» контроль над распределением внимания,
  что особенно важно в задачах с тонкими контекстными различиями.
  При этом основные веса модели остаются замороженными, а обучается лишь небольшой
  объём префиксных параметров

In [22]:
prefix_config = PrefixTuningConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    num_virtual_tokens=20,
    prefix_projection=True
)


In [23]:
base_model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
model = get_peft_model(base_model, prefix_config)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
model.print_trainable_parameters()

trainable params: 14,780,160 || all params: 124,267,014 || trainable%: 11.8939


In [25]:
training_args = TrainingArguments(
    output_dir="./prefix_tuning",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    weight_decay=0.01,
    seed=SEED,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset['validation'],
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [26]:
prefix_tuning_metrics = train_with_memory_and_metrics(trainer)

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 Macro
1,1.299300,1.189330,0.576500,0.197402,0.307434,0.237044
2,1.119100,1.019686,0.653500,0.307044,0.395256,0.343401
3,0.957200,0.857430,0.733000,0.455334,0.505883,0.467548
4,0.818500,0.714534,0.799500,0.683223,0.598204,0.563844
5,0.708900,0.585164,0.875000,0.863506,0.776042,0.801802
6,0.623200,0.516942,0.899000,0.874307,0.844231,0.853018
7,0.573000,0.473891,0.913500,0.897855,0.871263,0.881269
8,0.543200,0.447320,0.913500,0.877711,0.892905,0.884565
9,0.513000,0.429057,0.919000,0.898433,0.883951,0.889593
10,0.499400,0.419866,0.920500,0.899152,0.892237,0.892948


Количество обучаемых параметров: 14780160
Время: 712.65 s
Метрики: {'eval_loss': 0.39882901310920715, 'eval_accuracy': 0.9175, 'eval_precision': 0.8745803288345738, 'eval_recall': 0.8706279087832317, 'eval_f1_macro': 0.8714575933344596, 'eval_runtime': 2.2662, 'eval_samples_per_second': 882.53, 'eval_steps_per_second': 55.158, 'epoch': 15.0}
Использование памяти gpu: 1231.53 MB


## 5. LoRA

In [ ]:
r_values = [4, 8, 16, 32]
lora_experiments = []

for r in r_values:
    print(f"========= LoRA: r={r} =========")
    # 1) Базовая модель
    base = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,num_labels=num_labels)

    lora_cfg = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        inference_mode=False,
        r=r,
        lora_alpha=r * 4,      # часто alpha = r*X, где X из [1..4], чтобы было масштабирование адаптеров
        lora_dropout=0.1,
        target_modules=["query", "value"]
    )

    lora_model = get_peft_model(base, lora_cfg)
    lora_model.to(device)

    # 3) Trainer для LoRA
    args = TrainingArguments(
        output_dir=f"./lora_r{r}",
        eval_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=15,
        weight_decay=0.01,
        seed=SEED,)
    
    trainer = Trainer(
        model=lora_model,
        args=args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["validation"],
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    # 4) Запуск обучения + оценки
    res = train_with_memory_and_metrics(trainer)
    res["r"] = r
    lora_experiments.append(res)

========= LoRA: r=4 =========


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 Macro
1,1.508000,1.352425,0.527500,0.179383,0.281449,0.216507
2,1.191800,1.128044,0.577000,0.363874,0.308370,0.238922
3,1.076800,1.013305,0.603500,0.495377,0.352494,0.313696
4,0.945300,0.892368,0.648500,0.708553,0.454681,0.424758
5,0.861400,0.799344,0.682500,0.684464,0.500093,0.477441
6,0.803900,0.735382,0.711500,0.737918,0.536221,0.544251
7,0.742600,0.678475,0.743000,0.735738,0.592921,0.608466
8,0.700200,0.638714,0.760500,0.732017,0.634975,0.653188
9,0.677600,0.601813,0.778500,0.763902,0.658144,0.684174
10,0.638400,0.579892,0.790000,0.767874,0.677944,0.702255


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Количество обучаемых параметров: 152070
Время: 642.32 s
Метрики: {'eval_loss': 0.4835277795791626, 'eval_accuracy': 0.829, 'eval_precision': 0.7926230999087097, 'eval_recall': 0.7269297903006645, 'eval_f1_macro': 0.7512637486932152, 'eval_runtime': 2.1476, 'eval_samples_per_second': 931.271, 'eval_steps_per_second': 58.204, 'epoch': 15.0}
Использование памяти gpu: 1624.60 MB
========= LoRA: r=8 =========


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 Macro
1,1.221300,1.128124,0.583000,0.199886,0.311042,0.239805
2,1.041200,0.927677,0.637500,0.443091,0.409069,0.383253
3,0.800800,0.724905,0.732000,0.614309,0.560295,0.552399
4,0.665500,0.601161,0.775000,0.773729,0.631481,0.647141
5,0.577600,0.500710,0.828000,0.816004,0.733133,0.757751
6,0.509500,0.445828,0.843000,0.829358,0.746796,0.777206
7,0.450000,0.389147,0.871000,0.844545,0.818995,0.829622
8,0.433400,0.367373,0.887500,0.853812,0.860372,0.855201
9,0.391100,0.334151,0.889000,0.859636,0.850714,0.853827
10,0.372200,0.313420,0.898500,0.867644,0.866968,0.867079


Количество обучаемых параметров: 594438
Время: 643.63 s
Метрики: {'eval_loss': 0.2778737246990204, 'eval_accuracy': 0.9045, 'eval_precision': 0.8559403881670988, 'eval_recall': 0.8646255591482802, 'eval_f1_macro': 0.8595715039991082, 'eval_runtime': 2.1585, 'eval_samples_per_second': 926.58, 'eval_steps_per_second': 57.911, 'epoch': 15.0}
Использование памяти gpu: 1636.13 MB
========= LoRA: r=32 =========


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 Macro
1,1.161300,1.041651,0.603000,0.279227,0.338065,0.284661
2,0.873700,0.759423,0.722000,0.690683,0.523063,0.525195
3,0.668100,0.570412,0.810500,0.802108,0.695799,0.712243
4,0.553500,0.476883,0.845500,0.843893,0.733451,0.758296
5,0.469300,0.388643,0.879500,0.856225,0.821467,0.836462
6,0.395000,0.327499,0.906000,0.880971,0.859342,0.865470
7,0.344100,0.300885,0.910500,0.895904,0.870264,0.881512
8,0.330000,0.280700,0.916000,0.884613,0.894377,0.889169
9,0.289900,0.266553,0.920500,0.893514,0.890402,0.891752
10,0.280500,0.256258,0.920000,0.889338,0.895613,0.891733


Количество обучаемых параметров: 1184262
Время: 647.50 s
Метрики: {'eval_loss': 0.24380642175674438, 'eval_accuracy': 0.914, 'eval_precision': 0.8627140297875124, 'eval_recall': 0.8685216879504312, 'eval_f1_macro': 0.865518094629688, 'eval_runtime': 2.1585, 'eval_samples_per_second': 926.556, 'eval_steps_per_second': 57.91, 'epoch': 15.0}
Использование памяти gpu: 1642.61 MB


In [28]:
df_lora = pd.DataFrame([{
    "r": e["r"],
    "trainable_params_K": e["num_params"] / 1e3,
    "time_s": e["time_s"],
    "mem_MB": e["mem_mb"],
    "accuracy": e["metrics"]["eval_accuracy"],
    "precision": e["metrics"]["eval_precision"],
    "recall": e["metrics"]["eval_recall"],
    "f1_macro": e["metrics"]["eval_f1_macro"]
} for e in lora_experiments])
df_lora

,r,trainable_params_K,time_s,mem_MB,accuracy,precision,recall,f1_macro
0,4,152.070,642.315216,1624.602051,0.8290,0.792623,0.726930,0.751264
1,8,299.526,642.233332,1628.762207,0.8715,0.822131,0.803353,0.811113
2,16,594.438,643.630512,1636.133301,0.9045,0.855940,0.864626,0.859572
3,32,1184.262,647.497522,1642.613770,0.9140,0.862714,0.868522,0.865518


Подбирала ранги r {4,8,16,32}, лучший компромисс r=32, 
- несмотря на рост обучаемых параметров, время дообучения и занимаемая память не сильно растут в значениях по мере роста r, но метрики становятся значительно лучше

## 6. Сравнение

| Метод               | Параметров   | Память (MB) | Время (s) | Accuracy | Precision | Recall  | F1      |
|---------------------|--------------|-------------|-----------|----------|-----------|---------|---------|
| До finetuning       | n/a          | n/a         | n/a       | 0.151    | 0.1449    | 0.1619  | 0.0729  |
| Full finetuning     | 109 486 854  | 2153.35     | 1453.84   | 0.9295   | 0.8865    | 0.8853  | 0.8858  |
| Linear probing      |   299 910    | 1274.20     | 359.18    | 0.5030   | 0.4054    | 0.2717  | 0.2321  |
| Prefix tuning       | 14 780 160   | 1231.53     | 712.65    | 0.9175   | 0.8746    | 0.8706  | 0.8715  |
| LoRA (r=32)         | 1 184 262    | 1642.61     | 647.50    | 0.9140   | 0.8627    | 0.8685  | 0.8655  |

- Full FT даёт максимальное качество (F1 0.886, accuracy 0.929), но требует около 2 GB памяти и наибольшее количество времени 1450 s
- Prefix Tuning почти не уступает (F1 0.871) при halved памяти (1.2 GB) и времени (713 s).
- LoRA (r=32) обучает всего ~1 M параметров, сохраняя высокое качество (F1≈0.866) и средние ресурсы (~1.6 GB, 647 s).
- Linear Probing очень легковесен (300 k параметров, 359 s), но качество довольно низкое (F1 0.23).